In [28]:
from importlib import reload
import SABR_int as sabr
reload(sabr)
from typing import Optional, Union, Tuple, List
import numpy as np
import pandas as pd


In [22]:
class SABRCalibrator:
    """ Class for SABR model calibration and usage.
    
    Attributes:
        r(float): interest rate
        x0(np.ndarray): calibrated parameters of the SABR model, sabr_params = [sig_0, alpha, beta, rho]
        sabr(SABR): SABR-object 
    """
        
    def __init__(self, sabr: SABR, iv0, x0: list, fit_beta: bool, r: float = 0.0, beta: float = 0.5):
        """
            The __init__ method just save interest rate.
            
            Args:
                interest_rate(float): risk free interest rate, default value is zero
        """
        self.r = interest_rate
        self.x0 = x0
        self.sabr = sabr
        self.iv0 = iv0
        
        if not fit_beta:
            sabr_params[2] = 0.5
        
    def fit_iv(self, Niter:int=100) -> np.ndarray:
        """
            The fit_iv method calibrate pararmeters of the SABR model to market implied volatility
            Args:
                Niter(int): number of iteration
            Returns:
                fs(np.ndarray): array of errors on each iteration
        """
        def proj_sabr(sabr_params : np.ndarray)->np.ndarray:
            """
                This funciton project sabr parameters into valid range
                Attributes:
                    sabr_params(np.ndarray): model parameters

                Returns:
                    sabr_params(np.ndarray): clipped parameters
            """
            alpha, v, beta, rho = self.x0

            eps = 1e-6

            alpha = max(alpha, eps)
            v = max(v, eps)
            rho = np.clip(rho, -1 + eps, 1 - eps)
            beta = np.clip(beta, eps, 1 - eps)

            return np.asarray( [alpha, v, beta, rho] )

        def get_residals(sabr_params:np.ndarray) -> Tuple[ np.ndarray, np.ndarray ]:
            '''
                This function calculates residuals and Jacobian matrix
                Args:
                    sabr_params(np.ndarray): model params
                Returns:
                    res(np.ndarray) : vector or residuals
                    J(np.ndarray)   : Jacobian
            '''
            alpha, v, beta, rho = self.x0
            iv, iv_alpha, iv_v, iv_beta, iv_rho = self.sabr.sig, self.sabr.d_alpha, self.sabr.d_v, self.sabr.beta, self.sabr.rho
                
            res = iv - self.sabr.iv0
            J = np.asarray([iv_alpha, iv_v, iv_beta, iv_rho])
            return res * self.weights, J @ np.diag(self.weights)
        
        #optimization
        result = nonlinear_optimization(Niter, get_residals, proj_sabr, sabr_params)
        self.result = result['x']
        
        return result


            

In [29]:
data = pd.read_pickle("28.pkl")

In [30]:
data

,expiration,strike_price,exchange,symbol,local_timestamp,open_interest,last_price,bid_price,bid_amount,bid_iv,...,gamma,vega,theta,rho,tau,days,human_expiration,payoff,inverse_payoff,mid_price
141,1672387200000000,1300,deribit,ETH-30DEC22-1300-C,1669939197074952,31370.0,0.0730,0.0720,29.0,72.60,...,0.00153,1.41738,-1.83541,0.42604,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.07275
142,1672387200000000,1400,deribit,ETH-30DEC22-1400-C,1669939191456093,40458.0,0.0430,0.0425,42.0,70.85,...,0.00147,1.32299,-1.66698,0.30911,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.04325
143,1672387200000000,1500,deribit,ETH-30DEC22-1500-C,1669939193879451,27844.0,0.0255,0.0245,131.0,70.70,...,0.00122,1.09563,-1.38063,0.20929,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.02500
144,1672387200000000,1600,deribit,ETH-30DEC22-1600-C,1669939196476582,57516.0,0.0150,0.0145,179.0,72.08,...,0.00092,0.84633,-1.08870,0.13847,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.01500
145,1672387200000000,1700,deribit,ETH-30DEC22-1700-C,1669939196487709,32360.0,0.0095,0.0095,244.0,75.42,...,0.00067,0.64381,-0.86084,0.09386,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00975
146,1672387200000000,1800,deribit,ETH-30DEC22-1800-C,1669939196487937,44505.0,0.0065,0.0060,564.0,77.53,...,0.00049,0.48936,-0.68121,0.06519,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00650
147,1672387200000000,1900,deribit,ETH-30DEC22-1900-C,1669939196475515,37107.0,0.0045,0.0050,500.0,83.52,...,0.00038,0.40734,-0.60635,0.05136,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00525
148,1672387200000000,2000,deribit,ETH-30DEC22-2000-C,1669939196482592,54093.0,0.0040,0.0035,100.0,86.04,...,0.00029,0.32729,-0.50847,0.03904,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00400
149,1672387200000000,2100,deribit,ETH-30DEC22-2100-C,1669939196477084,28704.0,0.0030,0.0025,455.0,88.44,...,0.00023,0.25984,-0.41700,0.02945,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00300
150,1672387200000000,2200,deribit,ETH-30DEC22-2200-C,1669939196487355,33606.0,0.0025,0.0020,116.0,91.97,...,0.00018,0.22010,-0.36864,0.02408,0.077626,28,2022-12-30 11:00:00.000000,0.0,0.0,0.00250


In [31]:
sb = sabr.SABR(
    K = data.strike_price,
    F = data.underlying_index,
    T = data.tau,
    r = 0.0,
)

TypeError: __init__() missing 4 required positional arguments: 'alpha', 'v', 'beta', and 'rho'

In [21]:
import numpy as np
from typing import Tuple, Callable, Dict
from tqdm import tqdm

def nonlinear_optimization(Niter:int, 
                          f:Callable[ [np.ndarray], Tuple[np.ndarray, np.ndarray]], 
                          proj:Callable[ [np.ndarray], np.ndarray ], 
                          x0:np.ndarray) -> Dict:
    ''' 
        Nonlinear least squares method, Levenberg-Marquardt Method
        
        Args:
            Niter(int): number of iteration
            f(Callable[ [np.ndarray], Tuple[np.ndarray, np.ndarray]]): 
                callable, gets vector of model parameters x as input, 
                returns tuple res, J, where res is numpy vector of residues, 
                J is jacobian of residues with respect to x 
            proj(Callable[ [np.ndarray], np.ndarray ]):
                callable, gets vector of model parameters x,
                returns vector of projected parameters 
            x0(np.ndarray): initial parameters
        Returns:
            result(dict): dictionary with results
            result['xs'] contains optimized parameters on each iteration
            result['objective'] contains norm of residuals on each iteration
            result['x'] is optimized parameters
    '''
    x = x0.copy()

    

    mu = 100.0
    nu1 = 2.0
    nu2 = 2.0

    fs = []
    res, J = f(x)
    F = np.linalg.norm(res)
    
    result = { "xs":[x], "objective":[F], "x":None }
    
    for i in range(Niter):
        I = np.diag(np.diag(J @ J.T)) + 1e-5 * np.eye(len(x))
        dx = np.linalg.solve( mu * I + J @ J.T, J @ res )
        x_ = proj(x - dx)
        res_, J_ = f(x_)
        F_ = np.linalg.norm(res_)
        if F_ < F:
            x, F, res, J = x_, F_, res_, J_
            mu /= nu1
            result['xs'].append(x)
            result['objective'].append(F)
        else:
            i -= 1
            mu *= nu2
            continue        
        eps = 1e-10
        if F < eps:
            break
        result['x'] = x
    return result